# 🤖 ReAct Agent Demo Notebook

## Building Agentic AI with LangGraph & Google Gemini

This notebook demonstrates how to build a **ReAct Agent** that can:
- Think step-by-step
- Use tools to interact with the world
- Provide accurate, grounded responses

---

## 1. Setup & Installation

In [ ]:
# Install required packages
!pip install -q langchain langgraph langchain-google-genai

In [ ]:
# Set your API key
import os

# Option 1: Set directly (not recommended for production)
# os.environ["GOOGLE_API_KEY"] = "your-api-key-here"

# Option 2: Use getpass for secure input
from getpass import getpass
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass("Enter your Google API Key: ")

## 2. Understanding the ReAct Pattern

```
┌─────────────────────────────────────┐
│         ReAct Loop                   │
├─────────────────────────────────────┤
│   THOUGHT → ACTION → OBSERVATION    │
│       ↑          ↓                  │
│       └──────────┘                  │
│            ↓                        │
│        RESPONSE                     │
└─────────────────────────────────────┘
```

## 3. Define Tools

Tools are functions the agent can use to interact with the world.

In [ ]:
from langchain_core.tools import tool
import math
from datetime import datetime

# Tool 1: Calculator
@tool
def calculator(expression: str) -> str:
    """Evaluate a mathematical expression safely.
    
    Args:
        expression: Math expression like '2 + 2', 'sqrt(16)', '10 * 5'
    """
    safe_dict = {
        "sqrt": math.sqrt, "pow": math.pow,
        "sin": math.sin, "cos": math.cos,
        "pi": math.pi, "e": math.e,
        "abs": abs, "round": round,
    }
    try:
        result = eval(expression, {"__builtins__": {}}, safe_dict)
        return f"Result: {result}"
    except Exception as e:
        return f"Error: {str(e)}"

# Tool 2: Get Current Time
@tool
def get_current_time() -> str:
    """Get the current date and time."""
    now = datetime.now()
    return f"Current time: {now.strftime('%A, %B %d, %Y at %I:%M %p')}"

# Tool 3: Simple Search (Simulated)
@tool
def search(query: str) -> str:
    """Search for information on a topic.
    
    Args:
        query: The search query
    """
    # Simulated knowledge base
    knowledge = {
        "langgraph": "LangGraph is a library for building stateful AI agents with LLMs.",
        "react": "ReAct combines reasoning traces and actions in a synergistic loop.",
        "gemini": "Gemini is Google's most capable AI model family.",
    }
    
    for key, value in knowledge.items():
        if key in query.lower():
            return value
    return f"No specific information found for: {query}"

# Collect all tools
tools = [calculator, get_current_time, search]
print(f"✅ Defined {len(tools)} tools: {[t.name for t in tools]}")

## 4. Initialize the LLM (Gemini)

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

# Initialize Gemini
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.1,
)

# Bind tools to the LLM
llm_with_tools = llm.bind_tools(tools)

print("✅ LLM initialized with tools bound!")

## 5. Build the LangGraph Agent

In [ ]:
from typing import Annotated, TypedDict, Sequence
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage

# Define the state schema
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]

# System prompt
SYSTEM_PROMPT = """You are a helpful AI assistant using the ReAct pattern.
Think step-by-step, use tools when needed, and provide clear answers.
Available tools: calculator, search, get_current_time."""

# Agent node function
def agent_node(state: AgentState):
    messages = state["messages"]
    # Add system message if not present
    if not messages or not isinstance(messages[0], SystemMessage):
        messages = [SystemMessage(content=SYSTEM_PROMPT)] + list(messages)
    response = llm_with_tools.invoke(messages)
    return {"messages": [response]}

# Decision function
def should_continue(state: AgentState):
    last_message = state["messages"][-1]
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return "tools"
    return "end"

# Build the graph
graph = StateGraph(AgentState)
graph.add_node("agent", agent_node)
graph.add_node("tools", ToolNode(tools))
graph.set_entry_point("agent")
graph.add_conditional_edges("agent", should_continue, {"tools": "tools", "end": END})
graph.add_edge("tools", "agent")

# Compile
agent = graph.compile()

print("✅ Agent graph built!")

## 6. Visualize the Agent Graph

In [ ]:
# Display the graph structure (text representation)
print("""
Agent Graph Structure:
══════════════════════

    ┌─────────────┐
    │   START     │
    └──────┬──────┘
           │
           ▼
    ┌─────────────┐
    │   Agent     │◄─────────┐
    │   Node      │          │
    └──────┬──────┘          │
           │                 │
           ▼                 │
    ┌─────────────┐          │
    │  Decision   │          │
    │  (tools?)   │          │
    └──────┬──────┘          │
           │                 │
     ┌─────┴─────┐           │
     │           │           │
     ▼           ▼           │
┌─────────┐ ┌─────────┐      │
│  Tools  │ │   END   │      │
│  Node   │ └─────────┘      │
└────┬────┘                  │
     │                       │
     └───────────────────────┘
""")

## 7. Test the Agent!

In [ ]:
def run_agent(query: str, verbose: bool = True):
    """Run the agent with a query."""
    print(f"\n{'='*60}")
    print(f"❓ Query: {query}")
    print(f"{'='*60}\n")
    
    state = {"messages": [HumanMessage(content=query)]}
    
    if verbose:
        # Stream to show steps
        step_num = 1
        for step in agent.stream(state):
            for node_name, node_state in step.items():
                print(f"📌 Step {step_num} - Node: {node_name}")
                if "messages" in node_state:
                    for msg in node_state["messages"]:
                        msg_type = type(msg).__name__
                        content = msg.content[:100] + "..." if len(str(msg.content)) > 100 else msg.content
                        print(f"   └─ {msg_type}: {content}")
                        if hasattr(msg, "tool_calls") and msg.tool_calls:
                            for tc in msg.tool_calls:
                                print(f"      🔧 Tool Call: {tc['name']}({tc['args']})")
            step_num += 1
            print()
    
    # Get final result
    result = agent.invoke(state)
    final_response = result["messages"][-1].content
    
    print(f"{'─'*60}")
    print(f"✅ Final Answer: {final_response}")
    print(f"{'='*60}\n")
    
    return final_response

### Test 1: Math Problem

In [ ]:
run_agent("What is 25 multiplied by 4, then add 50?")

### Test 2: Information Query

In [ ]:
run_agent("What is LangGraph?")

### Test 3: Time Query

In [ ]:
run_agent("What is the current date and time?")

### Test 4: Complex Multi-Step

In [ ]:
run_agent("If I buy 3 items at $15.50 each and have a 10% discount, what's my final total?")

## 8. Interactive Mode

In [ ]:
# Interactive chat (run this cell and type your questions)
print("🎮 Interactive Mode - Type 'quit' to exit")
print("─" * 40)

while True:
    try:
        user_input = input("\n💬 You: ")
        if user_input.lower() in ['quit', 'exit', 'q']:
            print("\n👋 Goodbye!")
            break
        if user_input.strip():
            run_agent(user_input, verbose=False)
    except KeyboardInterrupt:
        print("\n👋 Session ended.")
        break

## 9. Summary

### What We Built:

1. **Tools** - Calculator, Search, Time
2. **LLM** - Google Gemini with tool binding
3. **Agent Graph** - LangGraph state machine
4. **ReAct Loop** - Think → Act → Observe

### Key Takeaways:

- 🤖 Agents can reason and use tools autonomously
- 🔄 LangGraph provides explicit control over the agent loop
- 🔧 Tools extend what agents can do
- 📝 System prompts guide agent behavior

### Next Steps:

- Add more tools (web search, databases)
- Implement memory for multi-turn conversations
- Add error handling and retries
- Deploy as an API service